In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import custom
from tqdm import tqdm
import numpy
import pickle
import pandas
from IPython.display import display

import torch
import torch.nn as nn
from torch.utils.data import DataLoader


In [2]:
with open("data/korean_word.pkl", mode = "rb") as f :
    word_dict = pickle.load(f)

with open("data/korean_vector_list.pkl", mode = "rb") as f :
    vec_list = pickle.load(f)

train_df = pandas.read_csv("data/korean_movie_train.txt", sep = "\t", encoding = "UTF8", index_col = 0)
test_df = pandas.read_csv("data/korean_movie_test.txt", sep = "\t", encoding = "UTF8", index_col = 0)

print(len(word_dict))
print(len(vec_list))
display(train_df)
display(test_df)

30187
30187


,document,label,word_len
0,아 아 더빙 . 진짜 짜증나 네요 목소리,0,8
1,흠 . 포스터 보고 초 딩 영화 줄 . 오버 연기 조차 가볍 지 않 구나,1,16
2,너무 재 밓 어 었 다 그래서 보 는 것 을 추천 하 ㄴ다,0,14
3,교도소 이야기 구 멀 ㄴ . 솔직히 재미 는 없 다 . 평점 조정,0,14
4,사이몬페그 의 익살 스럽 ㄴ 연기 가 돋보이 었 더 ㄴ 영화 ! 스파이 더 맨 에서...,1,36
...,...,...,...
149995,인간 이 문제지 . 소 는 뭔 죄 이 ㄴ가 .,0,11
149996,평점 이 너무 낮 아서 .,1,6
149997,이것 이 뭐 이 요 ? 한국 인은 거들먹거리 고 필리핀 혼혈 은 착하 다 ?,0,16
149998,청춘 영화 의 최고 봉 . 방황 과 우울하 었 더 ㄴ 날 들 의 자화상,1,16


,document,label,word_len
0,굳,1,1
2,뭐 야 이 평점 들 은 . 나쁘 지 는 않 지만 점 짜리 는 더더욱 아니 잖아,0,18
3,지루 하 지 는 않 은데 완전 막장 임 . 도 ㄴ 주고 보 기에 는 .,0,17
4,만 아니 었 어도 별 다섯 개 주 었 을 터 이 ㄴ데 . 왜 로 나오 아서 저 의 ...,0,28
5,음악 이 주가 되 ㄴ 최고 의 음악 영화,1,9
...,...,...,...
49995,오랜만 에 평점 로 긴하 었 네 킹 왕 짱 쌈 뽕 하 ㄴ 영화 를 만나 었 습니다 ...,1,26
49996,의지 박약 들 이나 하 는 거 이 다 탈영 은 일단 주인공 김 대희 닮 았 고 이등...,0,22
49997,그림 도 좋 고 완성도 도 높 았 지만 . 보 는 내내 불안 하 게 만들 ㄴ다,0,18
49998,절대 보 아서 는 안 되 ㄹ 영화 . 재미 도 없 고 기분 만 잡치 고 . 하 ㄴ ...,0,25


In [3]:
x = []
for i in tqdm(train_df.index) :
    x.append(custom.word_vectorize(train_df.loc[i, "document"], word_dict, 30))

t = train_df['label'].values.tolist()

print()
print(len(x))
print(len(t))

100%|███████████████████████████████████████████████████████████████████████| 148961/148961 [00:06<00:00, 22560.21it/s]


148961
148961


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(x, dtype = torch.long, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)

dataloader = DataLoader(list(zip(tensor_x, tensor_t)),batch_size=300,shuffle=True)

In [6]:
##AI
class NN(nn.Module) :
    def __init__(self, embedding_tensor) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True, padding_idx=0)
        self.rnn = nn.LSTM(embedding_tensor.shape[1],10,batch_first = True)
        self.f = nn.Sequential(
            nn.Linear(10,2),
        )
    def forward(self, x) :
        x = self.embedding(x)
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x

embedding_tensor = torch.tensor(vec_list, dtype = torch.float)
F = NN(embedding_tensor)
F = F.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 2)
epoch = 30

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
        y = F(x)
        loss = loss_function(y, t)
        loss_sum += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_sum /= len(dataloader)
    print(f"epoch {e+1} | loss {loss_sum}")



epoch 1 | loss 0.5486394381019432
epoch 2 | loss 0.4736629937376295
epoch 3 | loss 0.45470039921267413
epoch 4 | loss 0.44047997553103885
epoch 5 | loss 0.42941365833253686
epoch 6 | loss 0.42085624304815555
epoch 7 | loss 0.41577230121048403
epoch 8 | loss 0.41039354322661814
epoch 9 | loss 0.4056810713270061
epoch 10 | loss 0.4007940282764089
epoch 11 | loss 0.3978389974091374
epoch 12 | loss 0.39493328626726715
epoch 13 | loss 0.3910159700713167
epoch 14 | loss 0.38834966344372845
epoch 15 | loss 0.3851695385737198
epoch 16 | loss 0.38263493080734007
epoch 17 | loss 0.3819276371472318
epoch 18 | loss 0.37925906026627215
epoch 19 | loss 0.3775785912570339
epoch 20 | loss 0.37637494609389505
epoch 21 | loss 0.3753693126576767
epoch 22 | loss 0.37447596705655456
epoch 23 | loss 0.37163681457460046
epoch 24 | loss 0.3715730817385123
epoch 25 | loss 0.37020565818253176
epoch 26 | loss 0.368854891486331
epoch 27 | loss 0.36780230913843426
epoch 28 | loss 0.3661491902661276
epoch 29 | loss

In [7]:
x = []
for i in tqdm(test_df.index) :
    x.append(custom.word_vectorize(test_df.loc[i, "document"], word_dict, 40))

t = test_df['label'].values.tolist()

print()
print(len(x))
print(len(t))

100%|█████████████████████████████████████████████████████████████████████████| 49619/49619 [00:02<00:00, 24492.78it/s]


49619
49619


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(x, dtype = torch.long, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)

test_dataloader = DataLoader(list(zip(tensor_x, tensor_t)),batch_size=1000,shuffle=True)

In [9]:
correct = 0
total = 0

for x, t in test_dataloader :
    y = F(x)
    correct += (y.argmax(dim=-1) == t).sum().item()
    total += len(x)

print(correct / total)

0.8175094217940708


In [10]:
torch.save(F.to("cpu"), "korean_movie.pt")